# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 12 2022 11:35AM,244071,10,8139154-BO1,"Citieffe, Inc.",Released
1,Aug 12 2022 11:35AM,244070,10,CTF0011298-BO,"Citieffe, Inc.",Released
2,Aug 12 2022 11:34AM,244068,10,Eme-102866,Emerginnova,Released
3,Aug 12 2022 11:21AM,244066,20,8169438-BO,"ACI Healthcare USA, Inc.",Released
4,Aug 12 2022 11:21AM,244066,20,8169448-FLS-BO,"ACI Healthcare USA, Inc.",Released
5,Aug 12 2022 11:21AM,244066,20,8169459-BO,"ACI Healthcare USA, Inc.",Released
6,Aug 12 2022 11:21AM,244066,20,8169439-BO,"ACI Healthcare USA, Inc.",Released
7,Aug 12 2022 11:21AM,244066,20,8169440-FLS-BO,"ACI Healthcare USA, Inc.",Released
8,Aug 12 2022 11:21AM,244066,20,8169445-BO,"ACI Healthcare USA, Inc.",Released
9,Aug 12 2022 11:21AM,244066,20,8169447-BO,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,244065,Released,1
43,244066,Released,44
44,244068,Released,1
45,244070,Released,1
46,244071,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
244065,NaN,NaN,NaN,1.0
244066,NaN,NaN,NaN,44.0
244068,NaN,NaN,NaN,1.0
244070,NaN,NaN,NaN,1.0
244071,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
243963,1.0,0.0,0.0,17.0
243968,0.0,0.0,0.0,1.0
243969,0.0,0.0,0.0,1.0
243971,0.0,0.0,0.0,1.0
243973,0.0,0.0,4.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
243963,1.0,0,0,17
243968,0.0,0,0,1
243969,0.0,0,0,1
243971,0.0,0,0,1
243973,0.0,0,4,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,243963,1.0,0,0,17
1,243968,0.0,0,0,1
2,243969,0.0,0,0,1
3,243971,0.0,0,0,1
4,243973,0.0,0,4,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,243963,1.0,,,17
1,243968,0.0,,,1
2,243969,0.0,,,1
3,243971,0.0,,,1
4,243973,0.0,,4,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 12 2022 11:35AM,244071,10,"Citieffe, Inc."
1,Aug 12 2022 11:35AM,244070,10,"Citieffe, Inc."
2,Aug 12 2022 11:34AM,244068,10,Emerginnova
3,Aug 12 2022 11:21AM,244066,20,"ACI Healthcare USA, Inc."
47,Aug 12 2022 11:16AM,244065,10,"Citieffe, Inc."
48,Aug 12 2022 11:10AM,244064,10,"Citieffe, Inc."
49,Aug 12 2022 10:37AM,244061,16,Sartorius Stedim Filters Inc.
50,Aug 12 2022 10:34AM,244060,16,Sartorius Stedim Filters Inc.
51,Aug 12 2022 10:30AM,244059,16,Sartorius Stedim Filters Inc.
52,Aug 12 2022 10:24AM,244057,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Aug 12 2022 11:35AM,244071,10,"Citieffe, Inc.",0.0,,,1
1,Aug 12 2022 11:35AM,244070,10,"Citieffe, Inc.",0.0,,,1
2,Aug 12 2022 11:34AM,244068,10,Emerginnova,0.0,,,1
3,Aug 12 2022 11:21AM,244066,20,"ACI Healthcare USA, Inc.",0.0,,,44
4,Aug 12 2022 11:16AM,244065,10,"Citieffe, Inc.",0.0,,,1
5,Aug 12 2022 11:10AM,244064,10,"Citieffe, Inc.",0.0,,,1
6,Aug 12 2022 10:37AM,244061,16,Sartorius Stedim Filters Inc.,0.0,,,1
7,Aug 12 2022 10:34AM,244060,16,Sartorius Stedim Filters Inc.,0.0,,,1
8,Aug 12 2022 10:30AM,244059,16,Sartorius Stedim Filters Inc.,0.0,,,1
9,Aug 12 2022 10:24AM,244057,19,ACG North America LLC,0.0,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 11:35AM,244071,10,"Citieffe, Inc.",1,,
1,Aug 12 2022 11:35AM,244070,10,"Citieffe, Inc.",1,,
2,Aug 12 2022 11:34AM,244068,10,Emerginnova,1,,
3,Aug 12 2022 11:21AM,244066,20,"ACI Healthcare USA, Inc.",44,,
4,Aug 12 2022 11:16AM,244065,10,"Citieffe, Inc.",1,,
5,Aug 12 2022 11:10AM,244064,10,"Citieffe, Inc.",1,,
6,Aug 12 2022 10:37AM,244061,16,Sartorius Stedim Filters Inc.,1,,
7,Aug 12 2022 10:34AM,244060,16,Sartorius Stedim Filters Inc.,1,,
8,Aug 12 2022 10:30AM,244059,16,Sartorius Stedim Filters Inc.,1,,
9,Aug 12 2022 10:24AM,244057,19,ACG North America LLC,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 11:35AM,244071,10,"Citieffe, Inc.",1,,
1,Aug 12 2022 11:35AM,244070,10,"Citieffe, Inc.",1,,
2,Aug 12 2022 11:34AM,244068,10,Emerginnova,1,,
3,Aug 12 2022 11:21AM,244066,20,"ACI Healthcare USA, Inc.",44,,
4,Aug 12 2022 11:16AM,244065,10,"Citieffe, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 11:35AM,244071,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Aug 12 2022 11:35AM,244070,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Aug 12 2022 11:34AM,244068,10,Emerginnova,1.0,NaN,NaN
3,Aug 12 2022 11:21AM,244066,20,"ACI Healthcare USA, Inc.",44.0,NaN,NaN
4,Aug 12 2022 11:16AM,244065,10,"Citieffe, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 11:35AM,244071,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Aug 12 2022 11:35AM,244070,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Aug 12 2022 11:34AM,244068,10,Emerginnova,1.0,0.0,0.0
3,Aug 12 2022 11:21AM,244066,20,"ACI Healthcare USA, Inc.",44.0,0.0,0.0
4,Aug 12 2022 11:16AM,244065,10,"Citieffe, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5124462,194.0,70.0,12.0
102,244014,1.0,0.0,0.0
15,488070,2.0,9.0,7.0
16,1952383,8.0,0.0,0.0
19,244057,0.0,1.0,0.0
20,976098,90.0,19.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5124462,194.0,70.0,12.0
1,102,244014,1.0,0.0,0.0
2,15,488070,2.0,9.0,7.0
3,16,1952383,8.0,0.0,0.0
4,19,244057,0.0,1.0,0.0
5,20,976098,90.0,19.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,194.0,70.0,12.0
1,102,1.0,0.0,0.0
2,15,2.0,9.0,7.0
3,16,8.0,0.0,0.0
4,19,0.0,1.0,0.0
5,20,90.0,19.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,194.0
1,102,Released,1.0
2,15,Released,2.0
3,16,Released,8.0
4,19,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,19,20
Status,,,,,,
Completed,12.0,0.0,7.0,0.0,0.0,0.0
Executing,70.0,0.0,9.0,0.0,1.0,19.0
Released,194.0,1.0,2.0,8.0,0.0,90.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,19,20
0,Completed,12.0,0.0,7.0,0.0,0.0,0.0
1,Executing,70.0,0.0,9.0,0.0,1.0,19.0
2,Released,194.0,1.0,2.0,8.0,0.0,90.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,19,20
0,Completed,12.0,0.0,7.0,0.0,0.0,0.0
1,Executing,70.0,0.0,9.0,0.0,1.0,19.0
2,Released,194.0,1.0,2.0,8.0,0.0,90.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()